In [1]:
import torch
import torch.nn as nn
import os
from datetime import datetime
import time
import random
import cv2
import pandas as pd
import numpy as np
import albumentations as A
import matplotlib.pyplot as plt
from albumentations.pytorch.transforms import ToTensorV2
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from glob import glob


from tqdm import tqdm

from utils import visualize, plot_data
from scipy.io import loadmat

from torch.utils.data.distributed import DistributedSampler
from torch.nn.parallel import DistributedDataParallel


SEED = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)

In [2]:
path = '/mnt/home/hheat/USERDIR/counting-bench/data'
train_images = path + '/images'
test_images = path + '/test_images/images'
anno = path + '/annotation'
density_maps = path + '/dmap_amr'

LOG_PARA = 1000

In [3]:
def get_train_transforms():
    return A.Compose(
        [
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            #A.Resize(360,640,interpolation=2),
            #A.RandomSizedCrop(min_max_height=(409, 512), height=409, width=512, p=1.0),
            #A.Cutout(num_holes=8, max_h_size=64, max_w_size=64, fill_value=0, p=1.0),
        ],
        #additional_targets={'image': 'image','image1': 'image'}
        #keypoint_params = A.KeypointParams(format='xy')
)

def get_train_image_only_transforms():
    return A.Compose(
        [
            #A.Resize(360,640),
            A.OneOf([
                A.HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit= 0.2, 
                                     val_shift_limit=0.2, p=0.9),
                A.RandomBrightnessContrast(brightness_limit=0.2, 
                                           contrast_limit=0.2, p=0.9),
            ],p=0.9),
            A.Blur(blur_limit=3,p=0.2),
            A.Normalize(mean=mean,std=std,p=1.0,max_pixel_value=1.0),
            ToTensorV2(p=1.0),
        ],
        additional_targets={'image': 'image'}
    )

def get_valid_trainsforms():
    return A.Compose(
        [
            #A.Resize(360,640,interpolation=2),
            A.Normalize(mean=mean,std=std,p=1.0,max_pixel_value=1.0),
            ToTensorV2(p=1.0),
        ]
    )

# def get_valid_image_only_transforms():
#     return A.Compose(
#         [
#             A.Resize(360,640),
#         ],
#         additional_targets={'image': 'image'}
#     )

mean = torch.tensor([0.4939, 0.4794, 0.4583])
std = torch.tensor([0.2177, 0.2134, 0.2144])

def denormalize(img):
    img = img * std[...,None,None] + mean[...,None,None]
    img = img.permute(1,2,0).cpu().numpy()
    return img

In [4]:
class Counting_Dataset(Dataset):
    def __init__(self,path,image_fnames,dmap_folder,gt_folder=None,transforms=None,mosaic=False,downsample=4):
        '''
            path: root path 
            image_fnames: path of images
            dmap_folder: density map folder, eg: /dmap
            gt_folder: gt folder, currently set to visdrone xml format, modify _get_gt_data() if needed
            transforms: iteratable, can be tuple / list ... etc
            mosaic: mix up image and density map to form a new image, set to false by default
            downsample: resize dmap
        '''
        super().__init__()
        self.path = path
        self.image_fnames = image_fnames
        self.dmap_folder = path + dmap_folder
        self.transforms = transforms
        self.mosaic = mosaic
        self.downsample = downsample
        self.gt_folder = gt_folder # test purpose
        
    def __len__(self):
        return len(self.image_fnames)
    
    def __getitem__(self,idx):
        image_id = self.image_fnames[idx]
        
        if self.mosaic and random.randint(0,1) < 0.5:
            image, density_map, gt_points = self._load_mosaic_image_and_density_map(idx)
        else:
            image, density_map, gt_points = self._load_image_and_density_map(idx)
        
        h,w = image.shape[0]//self.downsample, image.shape[1]//self.downsample
        image = cv2.resize(image,(w, h))
        density_map = cv2.resize(density_map,(w//(self.downsample*2),h//(self.downsample*2)))#,interpolation=cv2.INTER_NEAREST)
        
        # Warning: doesn't work for cutout, uncommet transform and make fix code to enable cutout
        # Reason: cutout doesn't apply to mask, so mask must be image. check 01a bottom for code
        if self.transforms:
            for tfms in self.transforms:
                aug = tfms(**{
                    'image': image,
                    'mask': density_map,
                    #'keypoints': gt_points
                })
                #image, density_map, gt_points = aug['image'], aug['mask'], aug['keypoints']
                image, density_map = aug['image'], aug['mask'] # issue with previous keypoints (albumentation?)
        
        
        return image, density_map, image_id, gt_points
        
    
    def _get_dmap_name(self,fn):
        mask_name = fn.split('/')[-1].split('.')[0]
        mask_path = self.dmap_folder + '/' + mask_name + '.npy'
        return mask_path
    
    def _load_image_and_density_map(self,idx):
        image_fname = self.image_fnames[idx]
        dmap_fname = self._get_dmap_name(image_fname)
        image = cv2.imread(image_fname)
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB).astype(np.float32)
        image = image/255.
        d_map = np.load(dmap_fname,allow_pickle=True)
        
        #sanity check gt
        _, points = self._get_gt_data(idx)
        # end sanity check
        
        return image, d_map, points
    
    def _load_mosaic_image_and_density_map(self,idx):
        image_1, dmap_1, points_1 = self._load_image_and_density_map(idx)
        while True:
            idx_2 = random.randint(0,len(self.image_fnames)-1)
            if idx != idx_2:
                break
        image_2, dmap_2, points_2 = self._load_image_and_density_map(idx_2)
        
        imsize = min(*image_1.shape[:2])
        xc,yc = [int(random.uniform(imsize*0.4,imsize*0.6)) for _ in range(2)]
        h,w = image_1.shape[0], image_1.shape[1]

        pos = random.randint(0,1)
        if pos == 0: #top left
            x1a,y1a,x2a,y2a = 0,0,xc,yc # img_1
            x1b,y1b,x2b,y2b = w-xc,h-yc,w,h # img_2
        elif pos == 1: # top right
            x1a,y1a,x2a,y2a = w-xc,0,w,yc
            x1b,y1b,x2b,y2b = 0,h-yc,xc,h
        elif pos == 2: # bottom left
            x1a,y1a,x2a,y2a = 0,h-yc,xc,h
            x1b,y1b,x2b,y2b = w-xc,0,w,yc
        elif pos == 3: # bottom right
            x1a,y1a,x2a,y2a = w-xc,h-yc,w,h
            x1b,y1b,x2b,y2b = 0,0,xc,yc
        
        new_image = image_1.copy()
        new_dmap = dmap_1.copy()
        new_image[y1a:y2a,x1a:x2a] = image_2[y1b:y2b,x1b:x2b]
        new_dmap[y1a:y2a,x1a:x2a] = dmap_2[y1b:y2b,x1b:x2b]
        
        #TODO: sanity check to see generate gt
        
        new_gt_points = self._get_mixed_gt_points(points_1,points_2,(x1a,y1a,x2a,y2a),(x1b,y1b,x2b,y2b),(h,w))
        
        return new_image, new_dmap, new_gt_points
    
    '''
    The follow section blocks are for sanity check 
    to compare dmap.sum() with gt points
    remove if needed
    '''
    def _get_mixed_gt_points(self,points_1,points_2,img_1_loc, img_2_loc,img_shape):
#         fn_1, points_1 = self._get_gt_data(idx_1)
#         fn_2, points_2 = self._get_gt_data(idx_2)
        x1a,y1a,x2a,y2a = img_1_loc
        x1b,y1b,x2b,y2b = img_2_loc
        h,w = img_shape
        
        result_boxes = []
        result_boxes.append(points_2)
        result_boxes = np.concatenate(result_boxes,0)
        padw = x1a-x1b
        pady = y1a-y1b

        result_boxes[:,0] += padw
        result_boxes[:,1] += pady

        np.clip(result_boxes[:,0],0,w,out=result_boxes[:,0])
        np.clip(result_boxes[:,1],0,h,out=result_boxes[:,1])
        result_boxes = result_boxes.astype(np.int32)

        result_boxes = result_boxes[np.where(result_boxes[:,0] * result_boxes[:,1] > 0)]
        result_boxes = result_boxes[np.where(result_boxes[:,0] < w)]
        result_boxes = result_boxes[np.where(result_boxes[:,1] < h)]
        
        boxes = []
        for (x,y) in points_1:
            if x >= x1a and x <= x2a and y >= y1a and y <= y2a:
                continue
            else:
                boxes.append((x,y))
        if len(boxes) == 0:
            return result_boxes
        return np.concatenate((boxes, result_boxes),axis=0)
    
    def _get_gt_data(self,idx):
        if not self.gt_folder:
            return (None,0)
        fn = self.image_fnames[idx]
        anno_path = self.path + self.gt_folder + '/' + fn.split('/')[-1].split('.')[0] + '.mat'
        test_data = loadmat(anno_path)
        points = test_data['annotation'].astype(int)
        return fn, points

In [5]:
# ADD LOG_PARA to density map

class Crop_Dataset(Counting_Dataset):
    def __init__(self,path,image_fnames,dmap_folder,gt_folder=None,transforms=None,mosaic=False,downsample=4,crop_size=512,method='train'):
        super().__init__(path,image_fnames,dmap_folder,gt_folder,transforms,mosaic,downsample)
        self.crop_size = crop_size
        if method not in ['train','valid']:
            raise Exception('Not Implement')
        self.method = method
    
    def __getitem__(self,idx):
        fn = self.image_fnames[idx]
        
        image,density_map,gt_points = self._load_image_and_density_map(idx)
        h,w = image.shape[0], image.shape[1]
        #image = cv2.resize(image,(w, h))
        
        
        if self.method == 'train':
            #h,w = image.shape[:2]
            i,j = self._random_crop(h,w,self.crop_size,self.crop_size)
            image = image[i:i+self.crop_size,j:j+self.crop_size]
            density_map = density_map[i:i+self.crop_size,j:j+self.crop_size]
            
            gt_points = gt_points - [j,i]
            mask = (gt_points[:,0] >=0 ) * (gt_points[:,0] <= self.crop_size) * (gt_points[:,1]>=0) * (gt_points[:,1]<=self.crop_size)
            gt_points = gt_points[mask]
            density_map = cv2.resize(density_map,(self.crop_size//self.downsample,self.crop_size//self.downsample))
            
        else:
            density_map = cv2.resize(density_map,(w//self.downsample,h//self.downsample))#,interpolation=cv2.INTER_NEAREST)
            #density_map = density_map[1:-1,:]
        
        if self.transforms:
            for tfms in self.transforms:
                aug = tfms(**{
                    'image': image,
                    'mask': density_map,
                    #'keypoints': gt_points
                })
                #image, density_map, gt_points = aug['image'], aug['mask'], aug['keypoints']
                image, density_map = aug['image'], aug['mask'] # issue with previous keypoints (albumentation?)
        return image, density_map*LOG_PARA, fn, gt_points
    
    def _random_crop(self, im_h, im_w, crop_h, crop_w):
        res_h = im_h - crop_h
        res_w = im_w - crop_w
        i = random.randint(0, res_h)
        j = random.randint(0, res_w)
        return i, j

In [6]:
train_fp = glob(train_images + '/*.jpg')
test_fp = glob(test_images + '/*.jpg')

In [7]:
split = int(len(train_fp) * 0.8)
train_fp[0:split][:10]

['/mnt/home/hheat/USERDIR/counting-bench/data/images/11_233.jpg',
 '/mnt/home/hheat/USERDIR/counting-bench/data/images/12_240.jpg',
 '/mnt/home/hheat/USERDIR/counting-bench/data/images/08_113.jpg',
 '/mnt/home/hheat/USERDIR/counting-bench/data/images/03_319.jpg',
 '/mnt/home/hheat/USERDIR/counting-bench/data/images/06_176.jpg',
 '/mnt/home/hheat/USERDIR/counting-bench/data/images/05_105.jpg',
 '/mnt/home/hheat/USERDIR/counting-bench/data/images/11_204.jpg',
 '/mnt/home/hheat/USERDIR/counting-bench/data/images/14_253.jpg',
 '/mnt/home/hheat/USERDIR/counting-bench/data/images/14_129.jpg',
 '/mnt/home/hheat/USERDIR/counting-bench/data/images/20_248.jpg']

In [8]:
train_dataset = Crop_Dataset(path=path,
                             image_fnames=train_fp[:split],dmap_folder='/dmap_amr',
                             gt_folder='/annotation',
                             transforms=[get_train_transforms(),get_train_image_only_transforms()],
                             downsample=1,
                             crop_size=784
                                )

valid_dataset = Crop_Dataset(path=path,
                             image_fnames=test_fp,dmap_folder='/dmap_amr',
                             gt_folder='/annotation',
                             transforms=[get_valid_trainsforms()],
                             method='valid',
                             downsample=1,
                             crop_size=784
                                )

In [9]:
class TrainGlobalConfig:
    num_workers = 16
    batch_size = 16
    n_epochs = 120
    lr = 0.0002

    folder = 'AMRNet-7.19-784'
    downsample = 1

    # -------------------
    verbose = True
    verbose_step = 1
    # -------------------

    # --------------------
    step_scheduler = True  # do scheduler.step after optimizer.step
    validation_scheduler = False  # do scheduler.step after validation stage loss

    SchedulerClass = torch.optim.lr_scheduler.OneCycleLR
    scheduler_params = dict(
        max_lr=1e-4,
        #total_steps = len(train_dataset) // 4 * n_epochs, # gradient accumulation
        epochs=n_epochs,
        steps_per_epoch=int(len(train_dataset) / batch_size),
        pct_start=0.2,
        anneal_strategy='cos', 
        final_div_factor=10**5
    )
    
#     SchedulerClass = torch.optim.lr_scheduler.ReduceLROnPlateau
#     scheduler_params = dict(
#         mode='min',
#         factor=0.5,
#         patience=1,
#         verbose=False, 
#         threshold=0.0001,
#         threshold_mode='abs',
#         cooldown=0, 
#         min_lr=1e-8,
#         eps=1e-08
#     )

In [10]:
import torch.nn.functional as F
from torchvision import models

class VGG16_LCM(nn.Module):
    def __init__(self, load_weights=True):
        super(VGG16_LCM, self).__init__()

        self.layer5 = self.VGG_make_layers([64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M',
                                            512, 512, 512, 'M', 512, 512, 512, 'M'])

        self.reg_layer = nn.Sequential(
            nn.Conv2d(512, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 1, 1),
            nn.AvgPool2d(2, 2),
        )

        if load_weights:
            mod = models.vgg16(pretrained=False)
            pretrain_path = './vgg16-397923af.pth'
            mod.load_state_dict(torch.load(pretrain_path))
            print("loaded pretrain model: " + pretrain_path)

            self._initialize_weights()
            self.layer5.load_state_dict(mod.features[0:31].state_dict())

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.normal_(m.weight, std=0.01)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
                
    @torch.cuda.amp.autocast()
    def forward(self, x):
        x = self.layer5(x)
        x = self.reg_layer(x)

        return x

    def VGG_make_layers(self, cfg, in_channels=3, batch_norm=False, dilation=1):
        d_rate = dilation
        layers = []
        for v in cfg:
            if v == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=d_rate, dilation=d_rate)
                if batch_norm:
                    layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
                else:
                    layers += [conv2d, nn.ReLU(inplace=True)]
                in_channels = v
        return nn.Sequential(*layers)

In [11]:
def VGG_make_layers(cfg, in_channels=3, batch_norm=False, dilation=1):
        d_rate = dilation
        layers = []
        for v in cfg:
            if v == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=d_rate, dilation=d_rate)
                if batch_norm:
                    layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
                else:
                    layers += [conv2d, nn.ReLU(inplace=True)]
                in_channels = v
        return nn.Sequential(*layers)
    
vgg = VGG_make_layers([64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M',
                                            512, 512, 512, 'M', 512, 512, 512, 'M'])

In [12]:
from collections import OrderedDict
class VGG16_LCM_REG(nn.Module):
    def __init__(self, load_weights=False, stage_num=[3,3,3], count_range=100, lambda_i=1., lambda_k=1.):
        super(VGG16_LCM_REG, self).__init__()

        # cfg
        self.stage_num = stage_num
        self.lambda_i = lambda_i
        self.lambda_k = lambda_k
        self.count_range = count_range
        self.multi_fuse = True
        self.soft_interval = True

        self.layer3 = self.VGG_make_layers([64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512])
        self.layer4 = self.VGG_make_layers(['M', 512, 512, 512], in_channels=512)
        self.layer5 = self.VGG_make_layers(['M', 512, 512, 512], in_channels=512)

        if self.multi_fuse:
            self.fuse_layer5 = DC_layer(level=0)
            self.fuse_layer4 = DC_layer(level=1)
            self.fuse_layer3 = DC_layer(level=2)

        self.count_layer5 = Count_layer(pool=2)
        self.count_layer4 = Count_layer(pool=4)
        self.count_layer3 = Count_layer(pool=8)
        
        if self.soft_interval:
            self.layer5_k = nn.Sequential(
                nn.Conv2d(512, 1, kernel_size=1),
                nn.Tanh(),
            )
            self.layer4_k = nn.Sequential(
                nn.Conv2d(512, 1, kernel_size=1),
                nn.Tanh(),
            )
            self.layer3_k = nn.Sequential(
                nn.Conv2d(512, 1, kernel_size=1),
                nn.Tanh(),
            )
        
            self.layer5_i = nn.Sequential(
                nn.Conv2d(512, self.stage_num[0], kernel_size=1),
                nn.Sigmoid(),
            )
            self.layer4_i = nn.Sequential(
                nn.Conv2d(512, self.stage_num[1], kernel_size=1),
                nn.Sigmoid(),
            )
            self.layer3_i = nn.Sequential(
                nn.Conv2d(512, self.stage_num[2], kernel_size=1),
                nn.Sigmoid(),
            )

        self.layer5_p = nn.Sequential(
            nn.Conv2d(512, self.stage_num[0], kernel_size=1),
            nn.ReLU(),
        )
        self.layer4_p = nn.Sequential(
            nn.Conv2d(512, self.stage_num[1], kernel_size=1),
            nn.ReLU(),
        )
        self.layer3_p = nn.Sequential(
            nn.Conv2d(512, self.stage_num[2], kernel_size=1),
            nn.ReLU(),
        )

        if load_weights:
            #self._initialize_weights()
            
            mod = models.vgg16(pretrained=False)
            pretrain_path = './vgg16-397923af.pth'
            mod.load_state_dict(torch.load(pretrain_path))

            new_state_dict = OrderedDict()
            for key, params in mod.features[0:23].state_dict().items():
                new_state_dict[key] = params
            self.layer3.load_state_dict(new_state_dict)

            new_state_dict = OrderedDict()
            for key, params in mod.features[23:30].state_dict().items():
                key = str(int(key[:2]) - 23) + key[2:]
                new_state_dict[key] = params
            self.layer4.load_state_dict(new_state_dict)

            new_state_dict = OrderedDict()
            for key, params in mod.features[23:30].state_dict().items():
                key = str(int(key[:2]) - 23) + key[2:]
                new_state_dict[key] = params
            self.layer5.load_state_dict(new_state_dict)

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.normal_(m.weight, std=0.01)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        
        x3 = self.layer3(x)
        x4 = self.layer4(x3)
        x5 = self.layer5(x4)

        if self.multi_fuse:
            x5 = self.fuse_layer5(x5)
            x4 = self.fuse_layer4(x4)
            x3 = self.fuse_layer3(x3)

        x5_= self.count_layer5(x5)
        p5 = self.layer5_p(x5_)
        if self.soft_interval:
            k5 = self.layer5_k(x5_)
            i5 = self.layer5_i(x5_)

        x4_ = self.count_layer4(x4)
        p4 = self.layer4_p(x4_)
        if self.soft_interval:
            k4 = self.layer4_k(x4_)
            i4 = self.layer4_i(x4_)

        x3_ = self.count_layer3(x3)
        p3 = self.layer3_p(x3_)
        if self.soft_interval:
            k3 = self.layer3_k(x3_)
            i3 = self.layer3_i(x3_)

        stage1_regress = p5[:, 0, :, :] * 0
        stage2_regress = p4[:, 0, :, :] * 0
        stage3_regress = p3[:, 0, :, :] * 0

        for index in range(self.stage_num[0]):
            if self.soft_interval:
                stage1_regress = stage1_regress + (float(index) + self.lambda_i * i5[:, index, :, :]) * p5[:, index, :, :]
            else:
                stage1_regress = stage1_regress + float(index) * p5[:, index, :, :]
        stage1_regress = torch.unsqueeze(stage1_regress, 1)
        if self.soft_interval:
            stage1_regress = stage1_regress / ( float(self.stage_num[0]) * (1. + self.lambda_k * k5) )
        else:
            stage1_regress = stage1_regress / float(self.stage_num[0])


        for index in range(self.stage_num[1]):
            if self.soft_interval:
                stage2_regress = stage2_regress + (float(index) + self.lambda_i * i4[:, index, :, :]) * p4[:, index, :, :]
            else:
                stage2_regress = stage2_regress + float(index) * p4[:, index, :, :]
        stage2_regress = torch.unsqueeze(stage2_regress, 1)
        if self.soft_interval:
            stage2_regress = stage2_regress / ( (float(self.stage_num[0]) * (1. + self.lambda_k * k5)) *
                                                (float(self.stage_num[1]) * (1. + self.lambda_k * k4)) )
        else:
            stage2_regress = stage2_regress / float( self.stage_num[0] * self.stage_num[1] )


        for index in range(self.stage_num[2]):
            if self.soft_interval:
                stage3_regress = stage3_regress + (float(index) + self.lambda_i * i3[:, index, :, :]) * p3[:, index, :, :]
            else:
                stage3_regress = stage3_regress + float(index) * p3[:, index, :, :]
        stage3_regress = torch.unsqueeze(stage3_regress, 1)
        if self.soft_interval:
            stage3_regress = stage3_regress / ( (float(self.stage_num[0]) * (1. + self.lambda_k * k5)) *
                                                (float(self.stage_num[1]) * (1. + self.lambda_k * k4)) *
                                                (float(self.stage_num[2]) * (1. + self.lambda_k * k3)) )
        else:
            stage3_regress = stage3_regress / float( self.stage_num[0] * self.stage_num[1] * self.stage_num[2] )

        # regress_count = stage1_regress * self.count_range
        # regress_count = (stage1_regress + stage2_regress) * self.count_range
        regress_count = (stage1_regress + stage2_regress + stage3_regress) * self.count_range

        return regress_count

    def VGG_make_layers(self, cfg, in_channels=3, batch_norm=False, dilation=1):
        d_rate = dilation
        layers = []
        for v in cfg:
            if v == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=d_rate, dilation=d_rate)
                if batch_norm:
                    layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
                else:
                    layers += [conv2d, nn.ReLU(inplace=True)]
                in_channels = v
        return nn.Sequential(*layers)

class Count_layer(nn.Module):
    def __init__(self, inplanes=512, pool=2):
        super(Count_layer, self).__init__()
        self.avgpool_layer = nn.Sequential(
            nn.Conv2d(inplanes, inplanes, kernel_size=1),
            nn.ReLU(inplace=True),
            nn.AvgPool2d((pool, pool), stride=pool),
        )
        self.maxpool_layer = nn.Sequential(
            nn.Conv2d(inplanes, inplanes, kernel_size=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d((pool, pool), stride=pool),
        )
        self.conv1x1= nn.Sequential(
            nn.Conv2d(inplanes*2, inplanes, kernel_size=1),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        x_avg = self.avgpool_layer(x)
        x_max = self.maxpool_layer(x)

        x = torch.cat([x_avg, x_max], dim=1)
        x = self.conv1x1(x)
        return x


class DC_layer(nn.Module):
    def __init__(self, level, fuse=False):
        super(DC_layer, self).__init__()
        self.level = level
        self.conv1x1_d1 = nn.Conv2d(512, 512, kernel_size=1)
        self.conv1x1_d2 = nn.Conv2d(512, 512, kernel_size=1)
        self.conv1x1_d3 = nn.Conv2d(512, 512, kernel_size=1)
        self.conv1x1_d4 = nn.Conv2d(512, 512, kernel_size=1)

        self.conv_d1 = nn.Conv2d(512, 512, kernel_size=3, padding=1, dilation=1)
        self.conv_d2 = nn.Conv2d(512, 512, kernel_size=3, padding=2, dilation=2)
        self.conv_d3 = nn.Conv2d(512, 512, kernel_size=3, padding=3, dilation=3)
        self.conv_d4 = nn.Conv2d(512, 512, kernel_size=3, padding=4, dilation=4)
        
        self.fuse = fuse
        if self.fuse:
            self.fuse = nn.Conv2d(512*2, 512, kernel_size=3, padding=1)
            self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x1 = self.conv1x1_d1(x)
        x2 = self.conv1x1_d2(x)
        x3 = self.conv1x1_d3(x)
        x4 = self.conv1x1_d4(x)

        x1 = self.conv_d1(x1)
        x2 = self.conv_d2(x2)
        x3 = self.conv_d3(x3)
        x4 = self.conv_d4(x4)

        # x = torch.cat([x1, x2, x3, x4], dim=1)
        # x = self.relu(self.fuse(x))
        x = Maxout(x1, x2, x3, x4)
        return x

def Maxout(x1, x2, x3, x4):
    mask_1 = torch.ge(x1, x2)
    mask_1 = mask_1.float()
    x = mask_1 * x1 + (1-mask_1) * x2

    mask_2 = torch.ge(x, x3)
    mask_2 = mask_2.float()
    x = mask_2 * x + (1-mask_2) * x3

    mask_3 = torch.ge(x, x4)
    mask_3 = mask_3.float()
    x = mask_3 * x + (1-mask_3) * x4
    return x

In [13]:
#img, dmap, fn, points = train_dataset[250]
#kernel6=64
#filter6 = torch.ones(1, 1, kernel6, kernel6, requires_grad=False)
#gt_map_6 = F.conv2d(dmap.unsqueeze(0).unsqueeze(0), filter6, stride=kernel6)
#gt_map_6.shape
#mm = VGG16_LCM_REG(True)
#count_map = vgg(img.unsqueeze(0))
#print(count_map.shape)
#loss = nn.MSELoss()
#loss(count_map, gt_map_6)

In [14]:
def MSELoss_MCNN(preds,targs):
    return nn.MSELoss()(preds,targs)

def MAELoss_MCNN(preds,targs,upsample):
    return nn.L1Loss()((preds/LOG_PARA).sum(dim=[-1,-2])*upsample*upsample, (targs/LOG_PARA).sum(dim=[-1,-2])*upsample*upsample)

In [15]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [16]:
import warnings
warnings.filterwarnings("ignore")

#opt_level ='O1' # apex

class Fitter:
    
    def __init__(self, model, device, config):
        self.config = config
        self.epoch = 0

        self.base_dir = f'/mnt/home/zpengac/USERDIR/count/drone_benchmark/{config.folder}'
        if not os.path.exists(self.base_dir):
            os.makedirs(self.base_dir)
        
        self.log_path = f'{self.base_dir}/log.txt'
        self.best_summary_loss = 10**5

        self.model = model
        self.device = device

        param_optimizer = list(self.model.named_parameters())
        no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
        optimizer_grouped_parameters = [
            {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.001},
            {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ] 

        self.optimizer = torch.optim.AdamW(self.model.parameters(), lr=config.lr)
        
        #self.model, self.optimizer = amp.initialize(self.model,self.optimizer,opt_level=opt_level) # apex
        self.scaler = torch.cuda.amp.GradScaler()
        
        self.scheduler = config.SchedulerClass(self.optimizer, **config.scheduler_params)
        self.criterion = MSELoss_MCNN
        self.metric = MAELoss_MCNN
        self.log(f'Fitter prepared. Device is {self.device}')
        
        # self.iters_to_accumulate = 4 # gradient accumulation

    def fit(self, train_loader, validation_loader):
        for e in range(self.config.n_epochs):
            if self.config.verbose:
                lr = self.optimizer.param_groups[0]['lr']
                timestamp = datetime.utcnow().isoformat()
                self.log(f'\n{timestamp}\nLR: {lr}')

            t = time.time()
            summary_loss, mae_loss = self.train_one_epoch(train_loader)

            self.log(f'[RESULT]: Train. Epoch: {self.epoch}, mse_loss: {summary_loss.avg:.8f}, time: {(time.time() - t):.5f}')
            self.log(f'[RESULT]: Train. Epoch: {self.epoch}, mae_loss: {mae_loss.avg:.8f}, time: {(time.time() - t):.5f}')
            self.save(f'{self.base_dir}/last-checkpoint.bin')

            t = time.time()
            summary_loss, mae_loss = self.validation(validation_loader)

            self.log(f'[RESULT]: Val. Epoch: {self.epoch}, mse_loss: {summary_loss.avg:.8f}, time: {(time.time() - t):.5f}')
            self.log(f'[RESULT]: Val. Epoch: {self.epoch}, mae_loss: {mae_loss.avg:.8f}, time: {(time.time() - t):.5f}')
            if summary_loss.avg < self.best_summary_loss:
                self.best_summary_loss = summary_loss.avg
                self.model.eval()
                self.save(f'{self.base_dir}/best-checkpoint-{str(self.epoch).zfill(3)}epoch.bin')
                for path in sorted(glob(f'{self.base_dir}/best-checkpoint-*epoch.bin'))[:-3]:
                    os.remove(path)

            if self.config.validation_scheduler:
                self.scheduler.step(metrics=summary_loss.avg)

            self.epoch += 1

    def validation(self, val_loader):
        self.model.eval()
        summary_loss = AverageMeter()
        mae_loss = AverageMeter()
        t = time.time()
        for step, (images, density_maps, fns, gt_pts) in enumerate(val_loader):
            if self.config.verbose:
                if step % self.config.verbose_step == 0:
                    print(
                        f'Val Step {step}/{len(val_loader)}, ' + \
                        f'mse_loss: {summary_loss.avg:.8f}, ' + \
                        f'mae_loss: {mae_loss.avg:.8f}, ' + \
                        f'time: {(time.time() - t):.5f}', end='\r'
                    )
            with torch.no_grad():
                batch_size = images.shape[0]
                images = images.cuda().float()
                density_maps = density_maps.cuda().float()
                

                #preds = self.model(images)
                with torch.cuda.amp.autocast(): #native fp16
                    preds = self.model(images)
                    kernel6 = 64
                    filter6 = torch.ones(1, 1, kernel6, kernel6, requires_grad=False)
                    density_maps = F.conv2d(density_maps, filter6.cuda(), stride=kernel6)
                    loss = self.criterion(preds,density_maps/LOG_PARA)
                    metric_loss = self.metric(preds,density_maps,self.config.downsample)
                mae_loss.update(metric_loss.detach().item(),batch_size)
                summary_loss.update(loss.detach().item(), batch_size)
                
            #if step == 20:
            #    break

        return summary_loss, mae_loss

    def train_one_epoch(self, train_loader):
        self.model.train()
        summary_loss = AverageMeter()
        mae_loss = AverageMeter()
        t = time.time()
        for step, (images, density_maps, fns, gt_pts) in enumerate(train_loader):
            if self.config.verbose:
                if step % self.config.verbose_step == 0:
                    print(
                        f'Train Step {step}/{len(train_loader)}, ' + \
                        f'mse_loss: {summary_loss.avg:.8f}, ' + \
                        f'mae_loss: {mae_loss.avg:.8f}, ' + \
                        f'time: {(time.time() - t):.5f}', end='\r'
                    )
            
            images = images.cuda().float()
            batch_size = images.shape[0]
            density_maps = density_maps.cuda().float()
            
            
            self.optimizer.zero_grad()
            
            with torch.cuda.amp.autocast(): #native fp16
                preds = self.model(images)
                kernel6 = 64
                filter6 = torch.ones(1, 1, kernel6, kernel6, requires_grad=False)
                density_maps = F.conv2d(density_maps, filter6.cuda(), stride=kernel6).cuda()
                loss = self.criterion(preds,density_maps/LOG_PARA)
                metric_loss = self.metric(preds.detach(),density_maps.detach(),self.config.downsample)
            self.scaler.scale(loss).backward()
            
            # loss = loss / self.iters_to_accumulate # gradient accumulation
            
#             with amp.scale_loss(loss,self.optimizer) as scaled_loss: # apex
#                 scaled_loss.backward()
            #loss.backward()

            
            mae_loss.update(metric_loss.detach().item(),batch_size)
            summary_loss.update(loss.detach().item(), batch_size)
            
            #self.optimizer.step()
            self.scaler.step(self.optimizer) # native fp16
            
            if self.config.step_scheduler:
                self.scheduler.step()
            
            self.scaler.update() #native fp16
                
                
#             if (step+1) % self.iters_to_accumulate == 0: # gradient accumulation

#                 self.optimizer.step()
#                 self.optimizer.zero_grad()

#                 if self.config.step_scheduler:
#                     self.scheduler.step()
                    
            #if step == 20:
            #    break

        return summary_loss, mae_loss
    
    def save(self, path):
        self.model.eval()
        torch.save({
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'scheduler_state_dict': self.scheduler.state_dict(),
            'best_summary_loss': self.best_summary_loss,
            'epoch': self.epoch,
            #'amp': amp.state_dict() # apex
        }, path)

    def load(self, path):
        checkpoint = torch.load(path)
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        self.scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        self.best_summary_loss = checkpoint['best_summary_loss']
        self.epoch = checkpoint['epoch'] + 1
        
    def log(self, message):
        if self.config.verbose:
            print(message)
        with open(self.log_path, 'a+') as logger:
            logger.write(f'{message}\n')

In [17]:
def dist_init(host_addr, rank, local_rank, world_size, port=23456):
    host_addr_full = 'tcp://' + host_addr + ':' + str(port)
    torch.distributed.init_process_group("gloo", init_method=host_addr_full,
                                         rank=rank, world_size=world_size)
    assert torch.distributed.is_initialized()
    
def get_ip(iplist):
        ip = iplist.split('[')[0] + iplist.split('[')[1].split('-')[0]
        return ip

In [18]:
rank = int(os.environ['SLURM_PROCID'])
local_rank = int(os.environ['SLURM_LOCALID'])
world_size = int(os.environ['SLURM_NTASKS'])
iplist = os.environ['SLURM_JOB_NODELIST']
#ip = get_ip(iplist)
print(iplist, rank, local_rank, world_size)

gpu05 0 0 1


In [19]:
dist_init(iplist, rank, local_rank, world_size)

In [20]:
train_sampler = DistributedSampler(train_dataset, num_replicas=world_size, rank=rank)
val_sampler = DistributedSampler(valid_dataset, num_replicas=world_size, rank=rank)

In [21]:
def collate_fn(batch):
    imgs, dmaps, fns, gt_points = zip(*batch)
    imgs = torch.stack(imgs)
    dmaps = torch.stack(dmaps).unsqueeze(1)
    return imgs,dmaps,fns,gt_points

def run_training():
    device = torch.device('cuda:0')

    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=TrainGlobalConfig.batch_size,
        #sampler=RandomSampler(train_dataset),
        sampler=train_sampler,
        pin_memory=False,
        drop_last=True,
        num_workers=TrainGlobalConfig.num_workers,
        collate_fn=collate_fn,
    )

    val_loader = torch.utils.data.DataLoader(
        valid_dataset, 
        batch_size=TrainGlobalConfig.batch_size//4,
        num_workers=TrainGlobalConfig.num_workers//2,
        shuffle=False,
        #sampler=SequentialSampler(valid_dataset),
        sampler=val_sampler,
        pin_memory=True,
        collate_fn=collate_fn,
    )

    fitter = Fitter(model=net, device=device, config=TrainGlobalConfig)
#     fitter.load(f'{fitter.base_dir}/last-checkpoint.bin')
    fitter.fit(train_loader, val_loader)

In [22]:
net = VGG16_LCM_REG().cuda()
net = DistributedDataParallel(net)

In [24]:
run_training()

Fitter prepared. Device is cuda:0

2021-07-15T02:45:18.108357
LR: 3.9999999999999996e-05
[RESULT]: Train. Epoch: 0, mse_loss: 35.22208995, time: 67.39928 time: 41.24997
[RESULT]: Train. Epoch: 0, mae_loss: 10.52420766, time: 67.40969
[RESULT]: Val. Epoch: 0, mse_loss: 20.72305820, time: 26.64149, time: 24.28690
[RESULT]: Val. Epoch: 0, mae_loss: 131.89949799, time: 26.65091

2021-07-15T02:46:57.628189
LR: 4.013088592123436e-05
[RESULT]: Train. Epoch: 1, mse_loss: 26.28625470, time: 49.67701 time: 35.46557
[RESULT]: Train. Epoch: 1, mae_loss: 12.17845912, time: 49.68969
[RESULT]: Val. Epoch: 1, mse_loss: 20.15568488, time: 19.35900, time: 17.07036
[RESULT]: Val. Epoch: 1, mae_loss: 131.18062955, time: 19.36896

2021-07-15T02:48:11.707354
LR: 4.0523472305252516e-05
[RESULT]: Train. Epoch: 2, mse_loss: 18.89772458, time: 40.28414 time: 35.96966
[RESULT]: Train. Epoch: 2, mae_loss: 11.46640210, time: 40.29348
[RESULT]: Val. Epoch: 2, mse_loss: 20.53398686, time: 20.04821, time: 17.71468
[R

Exception in thread Thread-6679:
Traceback (most recent call last):
  File "/mnt/home/zpengac/.Miniconda3/envs/f4774e49c9ffe87fb0928ec97f8ff682/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/home/zpengac/.Miniconda3/envs/f4774e49c9ffe87fb0928ec97f8ff682/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/home/zpengac/.Miniconda3/envs/f4774e49c9ffe87fb0928ec97f8ff682/lib/python3.7/site-packages/torch/utils/data/_utils/pin_memory.py", line 25, in _pin_memory_loop
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
  File "/mnt/home/zpengac/.Miniconda3/envs/f4774e49c9ffe87fb0928ec97f8ff682/lib/python3.7/multiprocessing/queues.py", line 113, in get
    return _ForkingPickler.loads(res)
  File "/mnt/home/zpengac/.Miniconda3/envs/f4774e49c9ffe87fb0928ec97f8ff682/lib/python3.7/site-packages/torch/multiprocessing/reductions.py", line 282, in rebuild_storage_fd
    fd = df.detach()
  File "/mnt/home/

KeyboardInterrupt: 

In [23]:
val_loader = torch.utils.data.DataLoader(
        valid_dataset, 
        batch_size=16,
        num_workers=16,
        shuffle=False,
        #sampler=SequentialSampler(valid_dataset),
        sampler=val_sampler,
        pin_memory=True,
        collate_fn=collate_fn,
)

In [24]:
test_net = VGG16_LCM_REG().cuda()
test_net = DistributedDataParallel(test_net)

In [25]:
checkpoint = torch.load(f'/mnt/home/zpengac/USERDIR/count/drone_benchmark/AMRNet-7.26-784/best-checkpoint-113epoch.bin')
test_net.load_state_dict(checkpoint['model_state_dict'])
test_net.eval()

DistributedDataParallel(
  (module): VGG16_LCM_REG(
    (layer3): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(i

In [26]:
imgs, dmaps, fns, points = next(iter(val_loader))

In [27]:
with torch.no_grad():
    preds = test_net(imgs.cuda())

In [28]:
preds.shape

torch.Size([16, 1, 23, 42])

In [28]:
preds.sum(dim=[-1,-2])

tensor([[379.6073],
        [ 66.6977],
        [ 33.7065],
        [100.8357],
        [ 58.4938],
        [ 98.8577],
        [374.7237],
        [ 67.0916],
        [359.5981],
        [507.7969],
        [ 35.8292],
        [ 96.5278],
        [ 77.5106],
        [399.6143],
        [ 37.5311],
        [ 27.8512]], device='cuda:0')

In [29]:
dmaps.sum(dim=[-1,-2])/1000

tensor([[506.0000],
        [ 70.0000],
        [ 38.0000],
        [111.0000],
        [ 63.0000],
        [126.0000],
        [457.0000],
        [ 76.0000],
        [471.0000],
        [532.0000],
        [ 28.0000],
        [ 93.0000],
        [ 86.0000],
        [398.0000],
        [ 38.0000],
        [ 26.0000]], dtype=torch.float64)

In [30]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as pnsr

In [36]:
%%time

pre_count = []
gt_count = []
gt_points = []
avg_ssim = AverageMeter()
avg_pnsr = AverageMeter()
kernel6 = 64
for step, (imgs, dmaps, fns, points) in enumerate(val_loader):
    #with torch.cuda.amp.autocast():
    with torch.no_grad():
        imgs = imgs.cuda().float()
        preds = test_net(imgs)
    dmaps = dmaps / LOG_PARA

    filter6 = torch.ones(1, 1, kernel6, kernel6, requires_grad=False).float()
    dmaps = F.conv2d(dmaps.float(), filter6, stride=kernel6)
    
    for pred, dmap in zip(preds, dmaps):
        pred_array = pred.detach().cpu().numpy().squeeze()
        dmap_array = dmap.detach().cpu().numpy().squeeze()
        avg_ssim.update(ssim(dmap_array, pred_array, data_range=dmap_array.max()-dmap_array.min()))
        avg_pnsr.update(pnsr(dmap_array, pred_array, data_range=dmap_array.max()-dmap_array.min()))
    
    pre_count.extend(preds.sum(dim=[-1,-2]).detach().cpu().numpy())
    
    gt_count.extend(dmaps.sum(dim=[-1,-2]).detach().cpu().numpy())
    
    gt_p = []
    for p in points:
        gt_p.append(len(p))
    gt_points.extend(gt_p)

dmaps: torch.FloatTensor
preds: torch.cuda.FloatTensor
CPU times: user 5min 58s, sys: 11min 51s, total: 17min 49s
Wall time: 5min 4s


In [37]:
mae = mean_absolute_error(pre_count,gt_count)
mse = mean_squared_error(pre_count,gt_count)
nae = mae * len(pre_count) / np.sum(gt_count)

In [38]:
def count_parameters_in_MB(model):
    return np.sum(np.prod(v.size()) for name, v in model.named_parameters() if "auxiliary" not in name) / 1e6

print(f'#Paras: {count_parameters_in_MB(test_net)}')
print(f'MAE: {mae}, MSE: {mse}, NAE: {nae}')
print(f'SSIM: {avg_ssim.avg}, PNSR: {avg_pnsr.avg}')

#Paras: 56.424789
MAE: 12.378044128417969, MSE: 755.2483520507812, NAE: 0.09548808444236044
SSIM: 0.9011724980316669, PNSR: 29.211663611861038


In [37]:
a = torch.tensor([1,2])
a.type()

'torch.LongTensor'